In [ ]:
import sys
sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nhsmass.draw as draw
from nhsmass.spectrum import Spectrum
from nhsmass.brutto import brutto_gen
from nhsmass.spectrum import SpectrumList

#### Generate brutto formulas which we expect in samples

In [ ]:
gdf = brutto_gen(elems={'C':(4,41),'H':(4,81),'O':(0,26),'N':(0,4)})

#### Loading and assign brutto formules

In [ ]:
spec2 = Spectrum().read_csv(filename="data/sample2.csv",
                            mapper={'m/z':'mass', "I":'intensity'},
                            take_columns=['mass','intensity'],
                            sep=',',
                            )
spec2 = spec2.assign(generated_bruttos_table=gdf, rel_error=0.5)
draw.spectrum(spec2)
draw.vk(spec2)

#### Load assigned mass-list

In [ ]:
mapper = {"mw": "mass", "relativeAbundance": 'intensity'}
spec1 = Spectrum().read_csv("data/sample1.csv",
                            mapper = mapper,
                            sep=',',
                            ignore_columns=["peakNo", "errorPPM",'formula', "DBE", "class", "z"],
                            assign_mark=True
                           )
draw.spectrum(spec1)
draw.vk(spec1)

#### Search for common formulas

In [ ]:
common = spec1 & spec2
draw.vk(common)

#### Search for the symmetric difference

In [ ]:
common = spec1 ^ spec2
draw.vk(common)

In [ ]:
common = spec1 + spec2
draw.vk(common)

In [ ]:
common = spec1 - spec2
draw.vk(common)

#### Calculate simmetric indexes

In [ ]:
for index in ['cosine', 'tanimoto', 'jaccard']:
    value = spec1.simmilarity(spec2, mode=index)
    print(f'{index}: {round(value,3)} ')

Calculate DBE vs nO

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=75)

ab1 = spec1.get_dbe_vs_o(ax=ax, olim=(7,19))
ab2 = spec2.get_dbe_vs_o(ax=ax, olim=(7,19))

#### Also we can look for spec prepared by substraction by intensity and how its look

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=75)

spec1 = spec1.normalize()
spec2 = spec2.normalize()

sub1 = spec1.intens_sub(spec2)
sub2 = spec2.intens_sub(spec1)

sub1.get_dbe_vs_o(ax=ax, olim=(7,19))
sub2.get_dbe_vs_o(ax=ax, olim=(7,19))

it is rude, but we can consider that spec2 is more saturated.

#### Calculate molecular class zones and get density

In [ ]:
spec1_dens = spec1.get_mol_class()
spec2_dens = spec2.get_mol_class()
d = [spec1_dens, spec2_dens]

In [ ]:
specs = SpectrumList([spec1, spec2])
mol_density = pd.DataFrame()
for i, spec in enumerate(specs):
    mol_dens_spec = spec.get_mol_class()
    if i == 0:
       index = mol_dens_spec['class'].values
    mol_density[spec.metadata['name']] = mol_dens_spec['density']
mol_density.index = index
mol_density

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=75)

width=0.35

bot = np.array([0 for i in d])
labels = mol_density.columns

for key in mol_density.index:
    val = [mol_density.at[key, i] for i in labels]
    ax.bar(labels, val, width, label=key, bottom=bot)
    bot = bot + np.array(val)

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

or more simple way

In [ ]:
specs = SpectrumList([spec1, spec2])
specs.draw_mol_density()

#### Calc all

In [ ]:
spec1 = spec1.calc_all_metrics()
spec2 = spec2.calc_all_metrics()
spec1.table.info()

#### Plot DBE-O/C vs NOSC

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=100)

draw.scatter(spec1, x='NOSC', y='DBE-OC',ax=ax, color='blue', alpha=0.2)
draw.scatter(spec2, x='NOSC', y='DBE-OC',ax=ax, color='red', alpha=0.1)
ax.set_xlim(-1,1)
ax.set_ylim(-0.5,0.5)


In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=100)

draw.density(spec1, 'NOSC', color='blue', ax=ax)
draw.density(spec2, 'NOSC', color='red', ax=ax)


In [ ]:
draw.scatter(spec2, x='Ke', y='KMD', volume='intensity', size=3)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=100)

draw.vk(spec2, ax=ax, size_power=0.5, size=5)

In [ ]:
draw.vk(spec2, func=draw.scatter_density, title=False)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4), dpi=75)
draw.vk(spec2, func=draw.density_2D, ax=ax)

In [ ]:
spec2.squares_vk()

In [ ]:
spec1.get_mol_metrics()

In [ ]:
spec2.get_mol_metrics()

In [ ]:
specs = SpectrumList([spec1, spec2])

In [ ]:
specs.get_mol_metrics()